# 'Recommendation of similar articles from journal abstract analysis'  
# Modeling for recommendation creation
## 2019, Misty M. Giles
### https://github.com/OhThatMisty/astro_categories/

In [2]:
import gensim
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import LineSentence
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_short
import numpy as np
import os
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import spacy
import unicodedata

In [3]:
def normalize(text):
    '''Convert to ascii, remove special characters associated with LaTeX when given a df column,
       only keep alpha chars and contractions/posessives'''
    normalized_text = []
    
    for t in text:
        t = unicodedata.normalize('NFKD', t).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        # This line is necessary to separate some words since the LaTeX/math formatting is getting
        # quite a lot of attention in the w2v section.  
        t = re.sub('mathrm', ' ', t) 
        # Expand "not" before removing punctuation 
        t = re.sub('n\'t', ' not', t)
        t = strip_punctuation(t)
        # This line gets rid of non-alpha (mostly for digits now) 
        t = re.sub('[^A-Za-z]+', ' ', t) 
        normalized_text.append(strip_short(t))
    # strip_short gets rid of the rest of the math leftovers (and some abbreviations, like ir for
    # infrared - judged that the random letters left over from the math and units of measurement like
    # km, mm caused more noise than losing a few v short acronyms would cause problems - issue seen 
    # more frequently in tfidf)
    return normalized_text

# This function is to remove excess whitespace 
def remove(token):
    '''Provide feedback on whether a token is excess whitespace'''
    return token.is_space

# This function ensures that all printouts use the same formula
def join_tokens(sent):
    '''Joins tokens in a sent without whatever is in remove(), adds pronoun back
       in instead of -PRON-'''
    return ' '.join([token.lemma_ if token.lemma_ != '-PRON-' else token.text.lower()
                     for token in sent if not remove(token)])

# This function prevents nested lists that kill the vectorizer
def join_sentences(doc):
    '''Joins sentences in a doc (includes join_tokens)'''
    return ' '.join([join_tokens(sent) for sent in doc.sents])

# Set up spacy to lemmatize the text
nlp = spacy.load('en', disable=['ner'])

In [4]:
# Get the csv file created in the cleaning notebook
file = os.path.join('..', 'data', 'astro_intermediate.csv')
df = pd.read_csv(file, index_col=0)

# Set variables for testing speed
docs_to_run = 1000  # 1000 for testing, len(df) for real processing
train_docs = 800 if docs_to_run == 1000 else 50000
test_docs = docs_to_run - train_docs
train_docs, type(train_docs)

(800, int)

In [5]:
%%time

# Don't remove stopwords at this point.  Do that at the modeling stage with the models' 
# built-ins.  Screws up the phrasing.

# Clean the file (remove punctuation, lowercase, lemmatize, remove 1- and 2-char objects --
# most are math/LaTeX formatting leftovers or possessives)
text = [join_sentences(doc) for doc in nlp.pipe(normalize(df.abstract[:docs_to_run]), batch_size=1000)]

Wall time: 1min 38s


In [6]:
text[22:23]

['high resolution alma ghz and vla ghz measurement have be use image continuum and spectral line emission from the inner region the nearby infrared luminous galaxy detect compact luminous continuum emission the core with brightness temperature the ghz continuum equally compact but fainter flux suggest that the continuum opaque wavelength imply very large column density and that emerge from hot dust with temperature sim vibrationally excited line hcn and hcn vib be see emission and resolve scale the hcn vib emission reveal north south nuclear velocity gradient with projected rotation velocity kms the bright hcn vib emission orient perpendicular the velocity gradient ground state line hcn hco and show complex line absorption and emission feature hcn and hco have red shift reversed cygni profile consistent with gas inflow sim km the absorption feature can traced from the north east into the nucleus contrast show blue shift line wing extend km suggest that dense and slow outflow hide behin

###  Now that the text has been prepared, it's time to choose a sample abstract from the set of abstracts that won't be used to train the models.  This is a proof-of-concept method for testing the recommendation engine that can be tested even with internet issues.

In [7]:
# Pick an article to function as the sample and print some attributes.  
# Abstract is unaltered from download (more human-readable but includes formatting).
article_idx = np.random.randint(0, test_docs)
print(df.title.iloc[(article_idx + train_docs)], '\n')
print(df.abstract.iloc[(article_idx + train_docs)], '\n')
print(df.terms.iloc[(article_idx + train_docs)])

General relativistic effects in the galaxy bias at second order 

The local galaxy bias formalism relies on the energy constraint equation at the formation time to relate the metric perturbation to the matter density contrast. In the Newtonian approximation, this relationship is linear, which allows us to specify the initial galaxy density as a function of local physical operators. In general relativity however, the relationship is intrinsically nonlinear and a modulation of the short-wavelength mode by the long-wavelength mode might be expected. We describe in detail how to obtain local coordinates where the coupling of the long- to the short-wavelength modes is removed through a change of coordinates (in the absence of the primordial non-Gaussianity). We derive the general-relativistic correction to the galaxy bias expansion at second order. The correction does not come from the modulation of small-scale clustering by the long-wavelength mode; instead, it arises from distortions of t

### First model: tfidf using sklearn

In [8]:
%%time

# Set up the model for vectorizing/calculating the similarity; words must appear 
# in at least 500 documents.  sklearn stopwords are used, as removing stopwords at
# the beginning proved to damage the ngram results.
tfidf = TfidfVectorizer(ngram_range=(1,3), min_df=0.01, stop_words='english')

# Transform/fit the training and test data to the model
tfidf_matrix = tfidf.fit_transform(text[:train_docs]).todense()
article_matrix = tfidf.transform(text[train_docs:]).todense()

# Create a df of the model's values
tfidf_df = pd.DataFrame(tfidf_matrix, columns=tfidf.get_feature_names())
article_df = pd.DataFrame(article_matrix, columns=tfidf.get_feature_names())

Wall time: 1.37 s


In [9]:
len(tfidf.get_feature_names())

1836

In [10]:
# Check the top features of an abstract from the dataset
top_features = tfidf_df.iloc[22]
top_features.sort_values(ascending=False)[:10]

continuum         0.274706
inflow            0.269927
emission          0.262478
line              0.227307
ghz               0.215898
compact           0.179151
north             0.170520
gradient          0.159100
column density    0.153772
column            0.146456
Name: 22, dtype: float64

#### Finding recommendations from tfidf and cosine similarity, using the sample abstract above.

In [11]:
# Compute the document similarities with sklearn linear_kernel.  Per sklearn,
# linear_kernal is faster than cosine_similarity for tfidf.
document_similarity = linear_kernel(article_df.iloc[article_idx:article_idx+1], tfidf_df).flatten()

# Get the indices for the 5 documents that have highest cosine similarity to the sample.
# If document_similarity == 1.0, then the abstract randomly selected matches an abstract in 
# the sample.  Abstracts were filtered for exact matches, so resubmisions with only a word
# or two changed could cause this.  Throw that match out.
related_indices = document_similarity.argsort()[:-7:-1]
for i in range(0, len(related_indices)):
    if related_indices[i] == 1.0:
        related_indices.drop([i], inplace=True)
related_indices

array([230, 332, 757, 739,  14, 200], dtype=int64)

In [12]:
# Create a df with the attributes of the 5 similar documents
related_abstracts = df[['abstract', 'title', 'terms']].iloc[related_indices]
related_abstracts['document_similarity'] = document_similarity[related_indices]

# Print out the most similar documents
related_abstracts

,abstract,title,terms,document_similarity
230,The gravitational waveform of a merging stella...,Higher order gravitational-wave modes with lik...,astro-ph.IM|astro-ph.HE|gr-qc,0.244862
332,"Wave dispersion in a pulsar plasma (a 1D, stro...",Wave dispersion in pulsar plasma: 1. Plasma re...,physics.plasm-ph|astro-ph.HE,0.226747
757,The equipartition of magnetic and thermal ener...,Identification of magnetosonic modes in Galact...,physics.plasm-ph|astro-ph.GA,0.220757
739,"In the standard inflationary paradigm, cosmolo...",In search of an observational quantum signatur...,gr-qc|astro-ph.CO|quant-ph,0.181183
14,The fact that the spatial nonlocality of galax...,A new scale in the bias expansion,astro-ph.CO|astro-ph.GA,0.180749
200,We analyse the Planck full-mission cosmic micr...,Planck 2018 results. IX. Constraints on primor...,astro-ph.CO|gr-qc|hep-ph|hep-th,0.174493


In [13]:
# Get the features of the sample article
sample_features = article_df.iloc[article_idx]
sample_features.sort_values(ascending=False)[:10]

mode            0.377844
wavelength      0.363548
long            0.262896
bias            0.257492
modulation      0.223260
local           0.221206
relationship    0.208303
correction      0.186903
short           0.151912
galaxy          0.149452
Name: 112, dtype: float64

In [14]:
# Get the features for the highest-ranked related article
related_features = tfidf_df.iloc[related_indices[0]]
related_features.sort_values(ascending=False)[:10]

mode             0.504678
high order       0.447306
order            0.424577
bayesian         0.185420
high             0.160454
inference        0.143651
lead             0.135658
binary           0.127623
gravitational    0.118445
method           0.116768
Name: 230, dtype: float64

In [15]:
# Get the abstract for the highest-ranked related article
# Abstract is unaltered from download (more human-readable but includes formatting).
df.abstract[related_indices[0]]

'The gravitational waveform of a merging stellar-mass binary is described at leading order by a quadrupolar mode. However, the complete waveform includes higher-order modes, which encode valuable information not accessible from the leading-order mode alone. Despite this, the majority of astrophysical inferences so far obtained with observations of gravitational waves employ only the leading order mode because calculations with higher-order modes are often computationally challenging. We show how to efficiently incorporate higher-order modes into astrophysical inference calculations with a two step procedure. First, we carry out Bayesian parameter estimation using a computationally cheap leading-order-mode waveform, which provides an initial estimate of binary parameters. Second, we weight the initial estimate using higher-order mode waveforms in order to fold in the extra information from the full waveform. We use mock data to demonstrate the effectiveness of this method. We apply the 

###  Second model: gensim word2vec

In [16]:
%%time

# Set up the filepaths for the modeling
sentences_gensim = os.path.join('..', 'data', 'intermediate', 'sentences_gensim.txt')
abstracts_gensim = os.path.join('..', 'data', 'intermediate', 'abstracts_gensim.txt')
bi_sentences = os.path.join('..', 'data', 'intermediate', 'bi_sentences.txt')
tri_sentences = os.path.join('..', 'data', 'intermediate', 'tri_sentences.txt')

# Write the training sentences and the sample abstracts to a file so they can be streamed 
with open(sentences_gensim, 'w') as out_file:
     for sent in text[:train_docs]:
            out_file.write(sent + '\n')
            
with open(abstracts_gensim, 'w') as out_file:
     for sent in text[train_docs:]:
            out_file.write(sent + '\n')
            
# Set up streaming for the training data (create generator)
unigram_sentences = LineSentence(sentences_gensim)
bigram_sentences = LineSentence(bi_sentences)

Wall time: 29 ms


#### Set up and run the phrasing modes that will create bi- and trigrams.

In [17]:
%%time

# Use Phrases to create bigram model from the unigram sentences
biphrases = Phrases(unigram_sentences)
bigram_model = Phraser(biphrases)

# Write the bigram sentences
with open(bi_sentences, 'w') as out_file:
    for sent in unigram_sentences:
        sent_out = ' '.join(bigram_model[sent])
        out_file.write(sent_out + '\n')

# Use Phrases to create trigram model from the bigram sentences
triphrases = Phrases(bigram_sentences)
trigram_model = Phraser(triphrases)

# Write the trigram sentences
with open(tri_sentences, 'w') as out_file:
    for sent in bigram_sentences:
        sent_out = ' '.join(trigram_model[sent])
        # Phrasing finished; remove stopwords here
        out_file.write(remove_stopwords(sent_out) + '\n')

Wall time: 6.94 s


In [18]:
# Check the vocabulary sizes
len(biphrases.vocab), len(triphrases.vocab)

(77383, 80848)

In [19]:
# Testing with random words that stand a chance of being in the phrases
trigram_model[['black', 'hole', 'gravitational', 'wave', 'theory', 'gamma', 'ray', 'enrico', 'fermi', 'isaac', 'newton', 
              'fermi', 'gamma', 'ray', 'burst', 'monitor', 'ligo', 'interferometer', 'dark', 'matter', 'dark', 'energy']]

['black_hole',
 'gravitational_wave',
 'theory',
 'gamma_ray',
 'enrico',
 'fermi',
 'isaac',
 'newton',
 'fermi',
 'gamma_ray',
 'burst',
 'monitor',
 'ligo',
 'interferometer',
 'dark_matter',
 'dark_energy']

#### Set up the word2vec model and start training.

In [20]:
# Set up a staging directory for w2v
w2v_model_staging = os.path.join('..', 'data', 'intermediate', 'w2v_model_stage')

# Set up the generator for w2v
w2v_sentences = LineSentence(tri_sentences)

In [21]:
%%time

# Initiate the model and train neural net
# sg=0 is CBOW, sg=1 is skipgram
abstracts2vec = Word2Vec(w2v_sentences, size=100, window=5, min_count=10, sg=1, workers=4, iter=20)
abstracts2vec.save(w2v_model_staging)

# Load the finished model
abstracts2vec = Word2Vec.load(w2v_model_staging)
abstracts2vec.init_sims()

Wall time: 11.9 s


In [22]:
print(abstracts2vec.epochs, 'epochs')
print(abstracts2vec.corpus_count, 'abstracts')
print(abstracts2vec.wv.vectors.shape[0], 'words')
print(abstracts2vec.wv.vectors.shape[1], 'vectors')

20 epochs
800 abstracts
1744 words
100 vectors


In [23]:
abstracts2vec.wv.most_similar(positive='sun', topn=5)

[('distance', 0.6301395893096924),
 ('sim_kpc', 0.5542659759521484),
 ('euv_wave', 0.5490813255310059),
 ('nearly', 0.5418782234191895),
 ('north', 0.540489912033081)]

In [24]:
abstracts2vec.wv.most_similar(positive='ligo', topn=5)

[('electromagnetic_counterpart', 0.8779491186141968),
 ('advanced', 0.8622809648513794),
 ('virgo', 0.853947103023529),
 ('binary_neutron_star_merger', 0.8268074989318848),
 ('collaboration', 0.7956525087356567)]

In [25]:
abstracts2vec.wv.most_similar(positive='cosmology', topn=5)

[('dark_energy', 0.7869243621826172),
 ('dark_energy_model', 0.7838229537010193),
 ('modify_gravity', 0.7807337641716003),
 ('distinguish', 0.7380958795547485),
 ('cosmological_constant', 0.733410120010376)]

In [26]:
abstracts2vec.wv.most_similar(positive='rotational', topn=5)

[('heating', 0.7054296135902405),
 ('inner_disk', 0.6626993417739868),
 ('keplerian', 0.6554714441299438),
 ('lifetime', 0.6469238996505737),
 ('angular_momentum', 0.6453890800476074)]

In [27]:
abstracts2vec.wv.most_similar(positive=['sun', 'earth'], topn=5)

[('life', 0.6789458990097046),
 ('comet', 0.6759127378463745),
 ('sized', 0.6626361608505249),
 ('solar_system', 0.657239556312561),
 ('terrestrial', 0.6457377672195435)]

In [28]:
abstracts2vec.wv.most_similar(positive=['ligo', 'fermi'], topn=5)

[('lat', 0.8250271081924438),
 ('electromagnetic_counterpart', 0.8219002485275269),
 ('advanced', 0.7894283533096313),
 ('binary_neutron_star_merger', 0.781348705291748),
 ('fast_radio', 0.7430184483528137)]

In [29]:
abstracts2vec.wv.most_similar(positive=['ligo', 'gamma'], topn=5)

[('continue', 0.7618107795715332),
 ('electromagnetic_counterpart', 0.7318164110183716),
 ('virgo', 0.7140432000160217),
 ('observational_constraint', 0.7047683000564575),
 ('binary_neutron_star_merger', 0.7031246423721313)]

In [30]:
abstracts2vec.wv.most_similar(positive=['ligo', 'counterpart'], topn=5)

[('binary_neutron_star_merger', 0.8669383525848389),
 ('virgo', 0.8581392765045166),
 ('electromagnetic_counterpart', 0.8285713195800781),
 ('collaboration', 0.8218762874603271),
 ('advanced', 0.8203611373901367)]

In [31]:
abstracts2vec.wv.most_similar(positive=['binary', 'neutron'], topn=5)

[('electromagnetic_counterpart', 0.7022133469581604),
 ('ligo', 0.6952662467956543),
 ('compact_object', 0.6912593841552734),
 ('binary_neutron_star_merger', 0.6666845679283142),
 ('compact_binary', 0.6530908346176147)]